In [1]:
import dataset
import tensorflow as tf

In [2]:
from model_xception_original import ModelXception

In [3]:
frame = ModelXception(num_of_class=5, batch_size=32, num_of_data = 500000)

get_metadata: processing (0/500000)...
get_metadata: processing (1000/500000)...
get_metadata: processing (2000/500000)...
get_metadata: processing (3000/500000)...
get_metadata: processing (4000/500000)...
get_metadata: processing (5000/500000)...
get_metadata: processing (6000/500000)...
get_metadata: processing (7000/500000)...
get_metadata: processing (8000/500000)...
get_metadata: processing (9000/500000)...
get_metadata: processing (10000/500000)...
get_metadata: processing (11000/500000)...
get_metadata: processing (12000/500000)...
get_metadata: processing (13000/500000)...
get_metadata: processing (14000/500000)...
get_metadata: processing (15000/500000)...
get_metadata: processing (16000/500000)...
get_metadata: processing (17000/500000)...
get_metadata: processing (18000/500000)...
get_metadata: processing (19000/500000)...
get_metadata: processing (20000/500000)...
get_metadata: processing (21000/500000)...
get_metadata: processing (22000/500000)...
get_metadata: processing

get_metadata: processing (190000/500000)...
get_metadata: processing (191000/500000)...
get_metadata: processing (192000/500000)...
get_metadata: processing (193000/500000)...
get_metadata: processing (194000/500000)...
get_metadata: processing (195000/500000)...
get_metadata: processing (196000/500000)...
get_metadata: processing (197000/500000)...
get_metadata: processing (198000/500000)...
get_metadata: processing (199000/500000)...
get_metadata: processing (200000/500000)...
get_metadata: processing (201000/500000)...
get_metadata: processing (202000/500000)...
get_metadata: processing (203000/500000)...
get_metadata: processing (204000/500000)...
get_metadata: processing (205000/500000)...
get_metadata: processing (206000/500000)...
get_metadata: processing (207000/500000)...
get_metadata: processing (208000/500000)...
get_metadata: processing (209000/500000)...
get_metadata: processing (210000/500000)...
get_metadata: processing (211000/500000)...
get_metadata: processing (212000

get_metadata: processing (378000/500000)...
get_metadata: processing (379000/500000)...
get_metadata: processing (380000/500000)...
get_metadata: processing (381000/500000)...
get_metadata: processing (382000/500000)...
get_metadata: processing (383000/500000)...
get_metadata: processing (384000/500000)...
get_metadata: processing (385000/500000)...
get_metadata: processing (386000/500000)...
get_metadata: processing (387000/500000)...
get_metadata: processing (388000/500000)...
get_metadata: processing (389000/500000)...
get_metadata: processing (390000/500000)...
get_metadata: processing (391000/500000)...
get_metadata: processing (392000/500000)...
get_metadata: processing (393000/500000)...
get_metadata: processing (394000/500000)...
get_metadata: processing (395000/500000)...
get_metadata: processing (396000/500000)...
get_metadata: processing (397000/500000)...
get_metadata: processing (398000/500000)...
get_metadata: processing (399000/500000)...
get_metadata: processing (400000

In [4]:
input_size = (299,299,3)

In [5]:
train_generator,valid_generator, test_generator  = frame.get_datagenerators()
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


Found 215732 validated image filenames belonging to 6 classes.
Found 53932 validated image filenames belonging to 6 classes.
Found 67416 validated image filenames.


In [6]:
from tensorflow.keras.applications import Xception
base_model = Xception(weights='imagenet',
                          include_top=False,
                          input_shape=input_size)

In [7]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout

In [8]:
from tensorflow.keras.layers import Flatten
model = base_model.output
model = Flatten()(model)
model = Dense(256, activation='relu')(model)
model = Dropout(0.5)(model)
predictions = Dense(frame.num_of_class + 1, activation='softmax')(model)

In [9]:
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
def print_model_properties(model, indices = 0):
     for i, layer in enumerate(model.layers[indices:]):
        print(f"Layer {i+indices} | Name: {layer.name} | Trainable: {layer.trainable}")

In [11]:
def change_trainable_layers(model, trainable_index):
    for layer in model.layers[:trainable_index]:
        layer.trainable = False
    for layer in model.layers[trainable_index:]:
        layer.trainable = True

In [12]:
change_trainable_layers(model, 126)

In [13]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.RMSprop(lr=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
import datetime
log_dir="../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(
            x=train_generator,
            y=None,
            batch_size=None,
            epochs=10,
            verbose=1,
            validation_data=valid_generator,
            shuffle=False,
            class_weight=None,
            sample_weight=None,
            initial_epoch=0,
            steps_per_epoch=STEP_SIZE_TRAIN,
            validation_steps=STEP_SIZE_VALID,
            validation_freq=1,
            max_queue_size=10,
            callbacks=[tensorboard_callback],
            #workers=4,
            #use_multiprocessing=False,
        )

Train for 6741 steps, validate for 1685 steps
Epoch 1/10
6741/6741 [==============================] - 8458s 1s/step - loss: 1.4228 - accuracy: 0.3681 - val_loss: 1.7085 - val_accuracy: 0.2978
Epoch 2/10
6741/6741 [==============================] - 8452s 1s/step - loss: 1.2842 - accuracy: 0.4104 - val_loss: 1.7409 - val_accuracy: 0.3108
Epoch 3/10
6741/6741 [==============================] - 8459s 1s/step - loss: 1.2434 - accuracy: 0.4304 - val_loss: 2.5222 - val_accuracy: 0.2912
Epoch 4/10
6741/6741 [==============================] - 7595s 1s/step - loss: 1.2031 - accuracy: 0.4487


KeyboardInterrupt: 

In [22]:
scores = model.evaluate_generator(test_generator,
            steps=STEP_SIZE_TEST,
            max_queue_size=32*8
                       )

IndexError: list index out of range

In [20]:
test_generator.batch_size

32

In [48]:
model.summary()



Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [54]:
tf.__version__

'2.0.0'

In [56]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7596447123479397318
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3526369399608852566
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2977208287188318314
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6188478681867523919
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [ ]:
model.save('path_to_my_model.h5')

In [58]:
%load_ext tensorboard

In [60]:
%tensorboard --logdir '../logs'

In [1]:
import tensorflow
#tensorflow.config.list_physical_devices('GPU')
tensorflow.test.is_built_with_cuda()

True

In [62]:
c

<tf.Tensor: id=91132, shape=(2, 2), dtype=float32, numpy=
array([[22., 28.],
       [49., 64.]], dtype=float32)>